In [8]:
#imports 
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, HoverTool, ColorBar, LabelSet
from bokeh.plotting import figure
import datetime
import numpy as np
from bokeh.palettes import Spectral as paletin #RdYlGn more conventional...
from scipy.stats import norm
#from random import randint
from bokeh.transform import linear_cmap

In [9]:
#Black Scholes function
def BlackScholes(price, eXprice, ifree, iDate, fDate, vol, div):
    
    #divide by 100 to make decimals instead of percentajes
    ifree, vol, div = ifree/100, vol/100, div/100 

    # expected life of option in years. I do not know why 360 is used, but
    # sticking with tradition...
    timeExp = (fDate - iDate).days/360
    
    #d1 and d2
    d1 = (np.log(price/eXprice)+timeExp*(ifree-div+(vol**2)/2))/(vol*np.sqrt(timeExp))
    d2 = (np.log(price/eXprice)+timeExp*(ifree-div-(vol**2)/2))/(vol*np.sqrt(timeExp))
    #d2 = d1-vol*np.sqrt(timeExp)

    #calls and puts prices
    call =   price*np.exp(-div*timeExp)*norm.cdf( d1, 0, 1)  - eXprice*np.exp(-ifree*timeExp)*norm.cdf( d2, 0, 1)
    put  = eXprice*np.exp(-ifree*timeExp)*norm.cdf(-d2, 0, 1)- price*np.exp(-div*timeExp)*norm.cdf(-d1, 0, 1) 
    
    return call, put



In [10]:
#colorines: color defining function
def colorines (minPre, maxPre, pre, bins):
    sep = (maxPre - minPre)/bins
    bines = np.arange(minPre, maxPre, sep)
    for i in range(len(bines)):
        if bines[i] >= pre:
            break
        continue
    return i

In [11]:
#Plotting parameters
#how many days and price divisions
subDays   = 2
subPrices = 1
#min and max Call prices to plot
minCall   = -8
maxCall   =  8
#reverse the palette
paleta = paletin[10][::-1]

In [12]:
# miscelaneous inputs, if percentages no percentage sign
ifree    = 2.42                        # interest rate %
div      = 2.05                        # dividends %
precio   = 295                     # stock price at initial date

In [13]:
# specific option data
exPrecio = 300.0                       # excercise price
iDate   = datetime.date(2019, 5, 14)   # initial date (year, month, day)
fDate   = datetime.date(2019, 7, 30)   # final date
exDate  = datetime.date(2019, 7, 30)   # excercise date
vol     = 11.3                         # volatility %
pagado  = False                        # True if option is bought (net debit), False if sold or written (net credit)
opPrecio = 6.4                         # price of option

In [14]:
#array of days
dias = (np.arange(iDate, fDate, subDays, dtype = 'datetime64[D]')).tolist()

#allowed price change up and down, like 0.6 = 60%
deltaPrecio = 0.06

precioAlto = precio * (1 + deltaPrecio)
precioBajo = precio * (1 - deltaPrecio)

#array of prices
precios = (np.arange(precioBajo, precioAlto, subPrices, dtype = 'int64')).tolist()

#array of prices for plotting
pBajo = [precioBajo ]* len(precios)
pAlto = [precioAlto] * len(precios)

#define plot
p1 = figure(plot_height = 350, toolbar_location = 'right', title = 'Initial Price: black - Exercise Price: blue', x_axis_type = 'datetime')
p1.xaxis.formatter = DatetimeTickFormatter(days = ["%Y-%m-%d"], months = ["%Y-%m"], years = ["%Y"],)
p1.xgrid.grid_line_color = 'gray'
p1.ygrid.grid_line_color = 'gray'
p1.y_range.start = precioBajo
p1.y_range.end   = precioAlto
p1.x_range.start = iDate
p1.x_range.end   = fDate

#plot call matrix
for j in range(len(precios)):
    for i in range(len(dias)):
        #place cell
        abajo  = precioBajo + j * subPrices
        arriba = abajo + subPrices
        izda   = iDate + datetime.timedelta(days = i*subDays)
        dera   = izda  + datetime.timedelta(days = subDays)
        #Black Scholes - calculate call and put prices for cell
        call, put = BlackScholes(abajo, exPrecio, ifree, izda, exDate, vol, div)
        #assign color to cell
        if pagado:
            colorin = colorines(minCall, maxCall, call - opPrecio, 10)
        else:
            colorin = colorines(minCall, maxCall, opPrecio - call, 10)
        #print('{:8.3f} {:8.3f} {:8.3f} {:%Y-%m-%d} {:2.0f}'.format(call, put, abajo, izda, colorin) )
        #finally plot cell with corresponding color
        cuadro    = p1.quad(bottom=[abajo], top=[arriba], left=[izda], right=[dera], color = paleta[colorin])

# tooltips...
# many things in here, the hbars support the tooltip. I have found no way to do this in the cells. 
#The bars are superimposed over the cells, but because of the low alpha they are not visible. 
#The parameter width controls the width of the bars, if they are too narrow or wide, change it
source1 = ColumnDataSource(data = dict(dias = dias, pBajo = dias))
ver     = p1.vbar(x = 'dias', top = 'pBajo', width = 100000000, source = source1, color = 'white', alpha = 0.01)
ver_hover = HoverTool(renderers=[ver], tooltips=[('date', '@dias{%F}'), ('Stock', '$y')],
                  formatters=dict(dias='datetime'), mode = 'vline')
p1.add_tools(ver_hover)

#initial price line
p1.line([iDate, fDate], [precio, precio], line_width = 2, color = 'black')

#exercise option price
p1.line([iDate, fDate], [exPrecio, exPrecio], line_width = 2, color = 'blue')

#add option color scale
mapper = linear_cmap(field_name = 'precios', palette = paleta, low = minCall, high = maxCall)
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
p1.add_layout(color_bar, 'right')

show(p1)